## 各种import

In [2]:
%matplotlib inline

import os, sys 
sys.path.append(os.path.abspath("../examples"))

import utils; reload(utils)
from utils import *

from __future__ import division, print_function

Using Theano backend.


## 从Keras导入数据

In [3]:
from keras.datasets import mnist
(trn_data, trn_res), (test_data, test_res) = mnist.load_data()
(trn_data.shape, trn_res.shape, test_data.shape, test_res.shape)

((60000L, 28L, 28L), (60000L,), (10000L, 28L, 28L), (10000L,))

## 处理数据

### One-Hot-Encoding

In [4]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())
trn_label = onehot(trn_res)
test_label = onehot(test_res)

print(trn_res[:5])
print(trn_label[:5])

[5 0 4 1 9]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


### 对图像加入新的维度
即灰度, 维度大小为1; 彩色图片则维度大小为3, 代表RGB, 3个通道

In [5]:
trn_data = np.expand_dims(trn_data, 1)
test_data = np.expand_dims(test_data, 1)
trn_data.shape

(60000L, 1L, 28L, 28L)

### 对输入图像进行正则化

In [6]:
input_mean = trn_data.mean().astype(np.float32)
input_std = trn_data.std().astype(np.float32)

def norm_input(x):
    return (x-input_mean)/input_std

### 将输入数据转化为batches

In [7]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

## 1. 简单神经网络模型

In [10]:
simpleNN = Sequential()
simpleNN.add(Lambda(norm_input, input_shape=(1, 28, 28)))
simpleNN.add(Flatten())
simpleNN.add(Dense(512, activation='relu'))
simpleNN.add(Dense(10, activation='softmax'))
# 这里要把optimizer=Adam() 改成为 Adam()才会比较好, 为什么?
simpleNN.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

C:\anaconda2\lib\site-packages\keras\layers\core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


### 1.1 直接训练

In [11]:
#simpleNN.optimizer.lr = 0.1
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 14s - loss: 0.1858 - acc: 0.9441 - val_loss: 0.1087 - val_acc: 0.9660


In [12]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=3)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 16s - loss: 0.0847 - acc: 0.9732 - val_loss: 0.0843 - val_acc: 0.9738
Epoch 2/3
60000/60000 [==============================] - 19s - loss: 0.0631 - acc: 0.9797 - val_loss: 0.0799 - val_acc: 0.9746
Epoch 3/3
60000/60000 [==============================] - 22s - loss: 0.0487 - acc: 0.9843 - val_loss: 0.0780 - val_acc: 0.9786


### 1.2 生成batch再训练

利用ImageDataGenerator(), 使用fit_generator而不是fit
注意fit_generator的文档中参数由于版本升级发生了变化

In [9]:
simpleNN.optimizer.lr = 0.1
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=1, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 18s - loss: 0.5881 - acc: 0.8114 - val_loss: 0.5546 - val_acc: 0.8269


In [11]:
simpleNN.optimizer.lr = 0.01
simpleNN.fit_generator(batches, trn_data.shape[0], nb_epoch=3, validation_data=test_batches, nb_val_samples=test_data.shape[0])

Epoch 1/1
60000/60000 [==============================] - 21s - loss: 0.5534 - acc: 0.8212 - val_loss: 0.5068 - val_acc: 0.8295


## 2. 类似vgg16的卷积神经网络模型

注意比较各类activation之间的关系, 只有最后一个线性层使用softmax使得结果One-Hot-Coded, 其余层使用relu

In [16]:
simpleCNN = Sequential([
    Lambda(norm_input, input_shape=(1, 28, 28), output_shape=(1, 28, 28)),
    Convolution2D(32, 3, 3, activation="relu"),
    Convolution2D(32, 3, 3, activation="relu"),
    MaxPooling2D(),
    Convolution2D(64, 3, 3, activation="relu"),
    Convolution2D(64, 3, 3, activation="relu"),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(10, activation="softmax")])
simpleCNN.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
simpleCNN.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 562s - loss: 0.0997 - acc: 0.9695 - val_loss: 0.0390 - val_acc: 0.9873


In [13]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [14]:
model = get_model()

In [9]:
model.fit(trn_data, trn_label, validation_data=(test_data, test_label), nb_epoch=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 456s - loss: 0.0990 - acc: 0.9688 - val_loss: 0.0325 - val_acc: 0.9899


In [15]:
model.fit_generator(batches, trn_label.shape[0], validation_data=test_batches, nb_val_samples=test_label.shape[0], nb_epoch=1)

Epoch 1/1
60000/60000 [==============================] - 454s - loss: 0.1158 - acc: 0.9643 - val_loss: 0.0462 - val_acc: 0.9839


## Data Augmentation

同样是利用ImageDataGenerator, 传入Data Augmentation的参数. 参数规定了随机对图片进行旋转、缩放等操作等角度、幅度范围

In [18]:
batch_size = 64
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, shear_range=0.5)
batches = gen.flow(trn_data, trn_label, batch_size=batch_size)
test_batches = gen.flow(test_data, test_label, batch_size=batch_size)

In [19]:
simpleCNN.fit_generator(batches, 60000, nb_epoch=1, validation_data=test_batches, nb_val_samples=10000)

Epoch 1/1
60000/60000 [==============================] - 734s - loss: 0.1187 - acc: 0.9638 - val_loss: 0.0741 - val_acc: 0.9784


## Batch Normalization

In [ ]:
bn_model = Sequential()